## Singleton

In [ ]:
class Singleton:
    _instancia = None

    def __new__(cls):
        if cls._instancia is None:
            cls._instancia = super().__new__(cls)
        return cls._instancia


## Factory

## Builder

In [3]:
class Pedido:
    def __init__(self, lineas):
        self.lineas = lineas

    def total(self):
        return sum(linea['precio'] * linea['cantidad'] for linea in self.lineas)

    def __str__(self):
        return f"Pedido con {len(self.lineas)} productos, total: {self.total():.2f} €"


class PedidoBuilder:
    def __init__(self):
        self._lineas = []

    def agregar_producto(self, descripcion, cantidad, precio):
        self._lineas.append({
            "descripcion": descripcion,
            "cantidad": cantidad,
            "precio": precio
        })
        return self

    def build(self):
        return Pedido(self._lineas)


In [5]:
builder = PedidoBuilder()
pedido = (
    builder
    .agregar_producto("Pan", 2, 1.5)
    .agregar_producto("Leche", 1, 2.0)
    .build()
)

print(pedido)
# → Pedido con 2 productos, total: 5.00 €


Pedido con 2 productos, total: 5.00 €


- El Builder separa la construcción del objeto de su representación o lógica interna.

- Puedes añadir validaciones en build() (por ejemplo, no permitir pedidos vacíos).

- Es ideal cuando los objetos tienen muchos campos opcionales, o se construyen en pasos.

Refactoriza el siguiente código para:

- Crear una fábrica de pagos.
- Usar un singleton para la configuración de API keys.

In [ ]:
def pagar(tipo, datos):
    if tipo == "paypal":
        return f"Pagando con PayPal: {datos}"
    elif tipo == "stripe":
        return f"Pagando con Stripe: {datos}"


Refactor: aplicando Factory + Singleton
1. Definimos una clase base y las estrategias de pago

In [6]:
from abc import ABC, abstractmethod

class Pago(ABC):
    @abstractmethod
    def procesar(self, datos):
        pass

class PagoPaypal(Pago):
    def procesar(self, datos):
        return f"[PayPal] Pago procesado con datos: {datos}"

class PagoStripe(Pago):
    def procesar(self, datos):
        return f"[Stripe] Pago procesado con datos: {datos}"


2. Creamos una fábrica de pagos


In [7]:
class FabricaPago:
    _mapeo = {
        "paypal": PagoPaypal,
        "stripe": PagoStripe
    }

    @staticmethod
    def crear(metodo: str) -> Pago:
        clase = FabricaPago._mapeo.get(metodo.lower())
        if not clase:
            raise ValueError(f"Método de pago no soportado: {metodo}")
        return clase()


3. Añadimos un Singleton para configuración

In [8]:
class ConfiguracionPago:
    _instancia = None

    def __new__(cls):
        if not cls._instancia:
            cls._instancia = super().__new__(cls)
            cls._instancia.api_keys = {
                "paypal": "API_KEY_PAYPAL",
                "stripe": "API_KEY_STRIPE"
            }
        return cls._instancia


4. Función principal pagar() usando la fábrica

In [9]:
def pagar(metodo: str, datos: dict):
    pago = FabricaPago.crear(metodo)
    config = ConfiguracionPago()
    datos["api_key"] = config.api_keys.get(metodo)
    return pago.procesar(datos)


5. Ejemplo de uso

In [ ]:
print(pagar("paypal", {"cliente": "Andrés", "importe": 42}))

print(pagar("stripe", {"cliente": "Marina", "importe": 19.99}))


[PayPal] Pago procesado con datos: {'cliente': 'Andrés', 'importe': 42, 'api_key': 'API_KEY_PAYPAL'}
[Stripe] Pago procesado con datos: {'cliente': 'Marina', 'importe': 19.99, 'api_key': 'API_KEY_STRIPE'}


Beneficios de la refactorización

- Fácil de extender: solo añades una nueva clase PagoBizum y la registras en la fábrica.

- Separación de responsabilidades: pagar() ya no sabe cómo se implementa cada método.

- Testable y limpio: puedes testear cada clase por separado.